<a href="https://colab.research.google.com/github/advik-7/Ai-OS/blob/main/Gmail_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib -qq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.5 MB/s eta 0:00:00


In [ ]:
import imaplib
import email
from email.header import decode_header
from datetime import datetime, timedelta
from email.utils import parsedate_to_datetime
from transformers import pipeline
import pytz  # Added for timezone handling
import os

def fetch_and_classify_emails(email_user, email_pass):
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)  # CPU
    imap_host = "imap.gmail.com"

    # Make time_24hrs_ago timezone-aware
    utc = pytz.UTC
    time_24hrs_ago = utc.localize(datetime.now() - timedelta(days=1))

    try:
        mail = imaplib.IMAP4_SSL(imap_host)
        mail.login(email_user, email_pass)
        mail.select("inbox")

        # Fetch emails since 24 hours ago
        since_date = time_24hrs_ago.strftime("%d-%b-%Y")
        status, messages = mail.search(None, f'SINCE {since_date}')
        if status != "OK":
            print("No messages found!")
            return

        email_ids = messages[0].split()

        print("Processing recent emails...\n")
        for email_id in email_ids:
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            if status != "OK":
                print(f"Failed to fetch email ID {email_id.decode()}")
                continue

            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    date_header = msg.get("Date")
                    email_date = parsedate_to_datetime(date_header) if date_header else None

                    # Skip emails without a date or older than 24 hours
                    if not email_date or email_date < time_24hrs_ago:
                        continue

                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")

                    from_ = msg.get("From")
                    body = ""
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            content_disposition = str(part.get("Content-Disposition"))
                            if content_type == "text/plain" and "attachment" not in content_disposition:
                                try:
                                    body = part.get_payload(decode=True).decode()
                                    break
                                except Exception as e:
                                    print(f"Error decoding body: {e}")
                    else:
                        try:
                            body = msg.get_payload(decode=True).decode()
                        except Exception as e:
                            print(f"Error decoding body: {e}")

                    # Classify the email body
                    classification = classifier(
                        body[:1024],  # Limit to 1024 characters for efficiency
                        candidate_labels=["Spam", "Not Spam"]
                    )
                    category = classification["labels"][0]

                    print(f"Subject: {subject}")
                    print(f"From: {from_}")
                    print(f"Date: {email_date}")
                    print(f"Category: {category}")

                    if category == "Not Spam":
                        print(f"Full Email Body:\n{body}")
                    print("-" * 50)

        mail.logout()

    except Exception as e:
        print(f"An error occurred: {e}")



In [ ]:
import sqlite3

# Create database and table
def setup_database():
    conn = sqlite3.connect("emails.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Emails (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        subject TEXT,
        sender TEXT,
        date TEXT,
        category TEXT,
        full_body TEXT
    )
    """)
    conn.commit()
    conn.close()

# Insert email into the database
def store_email(subject, sender, date, category, full_body):
    conn = sqlite3.connect("emails.db")
    cursor = conn.cursor()
    cursor.execute("""
    INSERT INTO Emails (subject, sender, date, category, full_body)
    VALUES (?, ?, ?, ?, ?)
    """, (subject, sender, date, category, full_body))
    conn.commit()
    conn.close()

# Example usage
emails = [
    {
        "subject": "New Developer Alert for letvikk",
        "sender": "Facebook <noreply@developers.facebook.com>",
        "date": "2025-01-03 03:12:34-08:00",
        "category": "Not Spam",
        "full_body": "letvikk switched to live mode. View notifications in app dashboard."
    },
    {
        "subject": "Did you just log in near Vaghodia on a new device?",
        "sender": "Facebook <security@facebookmail.com>",
        "date": "2025-01-03 05:13:33-08:00",
        "category": "Not Spam",
        "full_body": "Security alert: login near Vaghodia, India on Chrome. Take steps to secure your account."
    },
    {
        "subject": "Get to know someone new",
        "sender": "Hinge Team <hello@mail.hinge.co>",
        "date": "2025-01-03 14:31:01+00:00",
        "category": "Spam",
        "full_body": "Check out our suggestions for meeting new people on Hinge."
    }
]



In [ ]:

if email_user and email_pass:
    fetch_and_classify_emails(email_user, email_pass)
else:
    print("Please set the EMAIL_USER and EMAIL_PASS environment variables.")

setup_database()
for email in emails:
    store_email(email["subject"], email["sender"], email["date"], email["category"], email["full_body"])

print("Emails stored successfully.")


Device set to use cpu


Processing recent emails...

Subject: New Developer Alert for letvikk
From: "Facebook" <noreply@developers.facebook.com>
Date: 2025-01-03 03:12:34-08:00
Category: Not Spam
Full Email Body:
Visit app dashboard
https://developers.facebook.com/apps/1750747725659950/alerts/?ref=email&alert_id=1753844032016986


Hi Neh,

letvikk switched to live mode.

You can view this and other developer notifications related to your app in the app dashboard.

Thanks,
The Facebook team



This message was sent to scince5678@gmail.com at your request.
Meta Platforms, Inc., Attention: Community Support, 1 Meta Way, Menlo Park, CA 94025


--------------------------------------------------
Subject: New Developer Alert for letvikk
From: "Facebook" <noreply@developers.facebook.com>
Date: 2025-01-03 03:23:44-08:00
Category: Not Spam
Full Email Body:
Visit app dashboard
https://developers.facebook.com/apps/1750747725659950/alerts/?ref=email&alert_id=1753848998683156


Hi Neh,

letvikk switched to live mode.

You 

In [ ]:

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(receiver_email, subject, body):
    """
    Function to send an email using Gmail's SMTP server.

    Args:
    sender_email (str): Sender's Gmail address
    app_password (str): App password generated for Gmail
    receiver_email (str): Recipient's email address
    subject (str): Subject of the email
    body (str): Body text of the email
    """
    try:
        # Set up the MIME message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        # Connect to Gmail's SMTP server and send email
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Secure the connection

        # Login to the Gmail account
        server.login(sender_email, app_password)

        # Send the email
        text = msg.as_string()
        server.sendmail(sender_email, receiver_email, text)

        print(f"Email sent successfully to {receiver_email}!")

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        # Close the connection to the server
        server.quit()

In [ ]:
import sqlite3
def fetch_email_by_id(email_id):
    conn = sqlite3.connect("emails.db")
    cursor = conn.cursor()

    cursor.execute("SELECT sender FROM Emails WHERE id = ?", (email_id,))
    result = cursor.fetchone()
    conn.close()

    if result:
        return result[0]
    else:
        return None
def send_email(to_email, subject, body):
    print(f"Sending email to {to_email} with subject '{subject}' and body '{body}'")

def email_pipeline(email_id, subject, body):
    email = fetch_email_by_id(email_id)

    if email:
        send_email(email, subject, body)
        return "Email sent successfully"
    else:
        return "Email not found for the given identifier."

email_id = 2
subject = "Test Subject"
body = "This is the body of the email."

result = email_pipeline(email_id, subject, body)
print(result)


Sending email to Facebook <security@facebookmail.com> with subject 'Test Subject' and body 'This is the body of the email.'
Email sent successfully
